In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import librosa
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


In [38]:
# Configuración
AUDIO_DIR = Path("Audios")
OUTPUT_DIR = Path("ProcessedData")
OUTPUT_DIR.mkdir(exist_ok=True)

SAMPLE_RATE = 16000
N_MELS = 64
N_MFCC = 13
N_FFT = 400
HOP_LENGTH = 160

print(f"Audio DIR: {AUDIO_DIR.absolute()}")
print(f"Output DIR: {OUTPUT_DIR.absolute()}")

Audio DIR: d:\Desarrollo\Universidad\Septimo\IA\RedNeuronalVoz\ProcesamientoAudios\Audios
Output DIR: d:\Desarrollo\Universidad\Septimo\IA\RedNeuronalVoz\ProcesamientoAudios\ProcessedData


In [39]:
class AudioProcessor:
    """Procesador de audios con PyTorch - Extracción robusta de características"""
    
    def __init__(self, sr=SAMPLE_RATE, n_mels=N_MELS, n_mfcc=N_MFCC):
        self.sr = sr
        self.n_mels = n_mels
        self.n_mfcc = n_mfcc
    
    def load_audio(self, filepath):
        """Carga y normaliza audio"""
        try:
            audio, sr = librosa.load(str(filepath), sr=self.sr, mono=True)
            # Normalizar
            audio = audio / (np.max(np.abs(audio)) + 1e-7)
            return torch.FloatTensor(audio), sr
        except Exception as e:
            print(f"Error cargando {filepath}: {e}")
            return None, None
    
    def extract_mel_spectrogram(self, audio):
        """Extrae Mel-spectrogram (2D)"""
        if isinstance(audio, torch.Tensor):
            audio = audio.cpu().numpy()
        
        mel_spec = librosa.feature.melspectrogram(
            y=audio, sr=self.sr, n_mels=self.n_mels,
            n_fft=N_FFT, hop_length=HOP_LENGTH
        )
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        # Normalizar por frecuencia
        mel_spec = (mel_spec - mel_spec.mean(axis=1, keepdims=True)) / (mel_spec.std(axis=1, keepdims=True) + 1e-7)
        return torch.FloatTensor(mel_spec)
    
    def extract_mfcc(self, audio):
        """Extrae MFCC + deltas (2D)"""
        if isinstance(audio, torch.Tensor):
            audio = audio.cpu().numpy()
        
        mfcc = librosa.feature.mfcc(y=audio, sr=self.sr, n_mfcc=self.n_mfcc)
        delta = librosa.feature.delta(mfcc)
        delta_delta = librosa.feature.delta(mfcc, order=2)
        
        features = np.vstack([mfcc, delta, delta_delta])
        features = (features - features.mean(axis=1, keepdims=True)) / (features.std(axis=1, keepdims=True) + 1e-7)
        return torch.FloatTensor(features)
    
    def extract_embedding(self, audio):
        """Extrae embedding: promedio Y desv. est. temporales + estadísticas"""
        mel_spec = self.extract_mel_spectrogram(audio)  # (n_mels, time_steps)
        mfcc = self.extract_mfcc(audio)  # (n_mfcc*3, time_steps)
        
        # ESTADÍSTICAS TEMPORALES de Mel-spectrogram
        mel_mean = mel_spec.mean(dim=1)  # Promedio temporal (n_mels,)
        mel_std = mel_spec.std(dim=1)    # Desv. estándar temporal (n_mels,)
        mel_max = mel_spec.max(dim=1)[0] # Máximo temporal (n_mels,)
        
        # ESTADÍSTICAS TEMPORALES de MFCC
        mfcc_mean = mfcc.mean(dim=1)  # Promedio temporal
        mfcc_std = mfcc.std(dim=1)    # Desv. estándar temporal
        mfcc_max = mfcc.max(dim=1)[0] # Máximo temporal
        
        # Combinar: media, desv.est, máx para cada stream
        # Total: (64+64+64) + (39+39+39) = 192 + 117 = 309 dims, pero reducimos...
        embedding = torch.cat([
            mel_mean, mel_std, mel_max,
            mfcc_mean, mfcc_std, mfcc_max
        ])  # (64*3 + 39*3,) = (309,)
        
        # Normalizar a [0,1] en lugar de L2 para preservar magnitud
        embedding = (embedding - embedding.min()) / (embedding.max() - embedding.min() + 1e-7)
        
        return embedding

print("✓ AudioProcessor definido")

✓ AudioProcessor definido


In [40]:
class AudioAugmenter:
    """Augmentación de audios con PyTorch"""
    
    @staticmethod
    def add_noise(audio, noise_factor=0.005):
        """Añade ruido gaussiano"""
        noise = torch.randn_like(audio) * noise_factor
        return audio + noise
    
    @staticmethod
    def time_stretch(audio, rate=1.1):
        """Time stretching con librosa"""
        audio_np = audio.cpu().numpy() if isinstance(audio, torch.Tensor) else audio
        stretched = librosa.effects.time_stretch(audio_np, rate=rate)
        return torch.FloatTensor(stretched)
    
    @staticmethod
    def pitch_shift(audio, n_steps=2, sr=SAMPLE_RATE):
        """Pitch shifting con librosa"""
        audio_np = audio.cpu().numpy() if isinstance(audio, torch.Tensor) else audio
        shifted = librosa.effects.pitch_shift(audio_np, sr=sr, n_steps=n_steps)
        return torch.FloatTensor(shifted)
    
    @staticmethod
    def generate_augmentations(audio, num_aug=12):
        """Genera EXACTAMENTE 12 augmentaciones distintas y variadas"""
        augmentations = []
        
        # 1. Original
        augmentations.append(audio)
        
        # 2-4. Ruido en 3 niveles
        for noise_level in [0.003, 0.007, 0.012]:
            augmentations.append(AudioAugmenter.add_noise(audio, noise_level))
        
        # 5-6. Time stretch
        for rate in [0.92, 1.08]:
            try:
                augmentations.append(AudioAugmenter.time_stretch(audio, rate))
            except:
                augmentations.append(AudioAugmenter.add_noise(audio, 0.005))
        
        # 7-8. Pitch shift
        for steps in [-2, 2]:
            try:
                augmentations.append(AudioAugmenter.pitch_shift(audio, n_steps=steps))
            except:
                augmentations.append(AudioAugmenter.add_noise(audio, 0.008))
        
        # 9. Ruido + time stretch
        try:
            noisy = AudioAugmenter.add_noise(audio, 0.005)
            stretched = AudioAugmenter.time_stretch(noisy, rate=0.95)
            augmentations.append(stretched)
        except:
            augmentations.append(AudioAugmenter.add_noise(audio, 0.010))
        
        # 10. Ruido + pitch shift
        try:
            noisy = AudioAugmenter.add_noise(audio, 0.008)
            shifted = AudioAugmenter.pitch_shift(noisy, n_steps=1)
            augmentations.append(shifted)
        except:
            augmentations.append(AudioAugmenter.add_noise(audio, 0.012))
        
        # 11. Time stretch + pitch shift
        try:
            stretched = AudioAugmenter.time_stretch(audio, rate=1.05)
            shifted = AudioAugmenter.pitch_shift(stretched, n_steps=-1)
            augmentations.append(shifted)
        except:
            augmentations.append(AudioAugmenter.add_noise(audio, 0.006))
        
        # 12. Ruido + time stretch + pitch shift
        try:
            noisy = AudioAugmenter.add_noise(audio, 0.010)
            stretched = AudioAugmenter.time_stretch(noisy, rate=0.98)
            shifted = AudioAugmenter.pitch_shift(stretched, n_steps=1)
            augmentations.append(shifted)
        except:
            augmentations.append(AudioAugmenter.add_noise(audio, 0.015))
        
        return augmentations[:num_aug]

print("✓ AudioAugmenter definido")

✓ AudioAugmenter definido


In [41]:
# Explorar audios
personas = {}

if AUDIO_DIR.exists():
    for persona_dir in sorted(AUDIO_DIR.iterdir()):
        if persona_dir.is_dir():
            nombre = persona_dir.name
            archivos = list(persona_dir.glob("*.opus")) + \
                       list(persona_dir.glob("*.wav")) + \
                       list(persona_dir.glob("*.mp3"))
            if archivos:
                personas[nombre] = archivos
                print(f"👤 {nombre:20} - {len(archivos)} audios")

print(f"\n✓ Total: {len(personas)} usuarios, {sum(len(a) for a in personas.values())} archivos")

👤 AlisonSalas          - 3 audios
👤 FreddyAlvarez        - 2 audios
👤 IsmaelSailema        - 3 audios
👤 WilliamChimborazo    - 3 audios

✓ Total: 4 usuarios, 11 archivos


In [42]:
# Procesar audios
processor = AudioProcessor(sr=SAMPLE_RATE, n_mels=N_MELS, n_mfcc=N_MFCC)
augmenter = AudioAugmenter()

embeddings_por_usuario = {}

print("\n🔄 Procesando audios...")
print("=" * 70)

for nombre_usuario, archivos in personas.items():
    embeddings_usuario = []
    print(f"\n👤 {nombre_usuario}")
    
    for archivo in archivos:
        audio, sr = processor.load_audio(archivo)
        
        if audio is None:
            continue
        
        # Generar augmentaciones (12x para más datos)
        audios_variantes = augmenter.generate_augmentations(audio, num_aug=12)
        
        for audio_variant in audios_variantes:
            try:
                embedding = processor.extract_embedding(audio_variant)
                embeddings_usuario.append(embedding.numpy())
            except Exception as e:
                print(f"   Error: {e}")
                continue
        
        print(f"   ✓ {archivo.name} ({len(audios_variantes)} variantes)")
    
    if embeddings_usuario:
        embeddings_por_usuario[nombre_usuario] = np.array(embeddings_usuario)
        print(f"   → {len(embeddings_usuario)} embeddings (shape: {embeddings_por_usuario[nombre_usuario].shape})")

print(f"\n" + "=" * 70)
print(f"✓ Procesamiento completado")
for usuario, emb in embeddings_por_usuario.items():
    print(f"  {usuario:20} - {len(emb)} embeddings (dim: {emb.shape[1]})")


🔄 Procesando audios...

👤 AlisonSalas
   ✓ AlisonSalas3.opus (12 variantes)
   ✓ AlisonSalas4.opus (12 variantes)
   ✓ AlisonSalas5.opus (12 variantes)
   → 36 embeddings (shape: (36, 309))

👤 FreddyAlvarez
   ✓ FreddyAlvarez1.opus (12 variantes)
   ✓ FreddyAlvarez2.opus (12 variantes)
   → 24 embeddings (shape: (24, 309))

👤 IsmaelSailema
   ✓ IsmaelSailema.opus (12 variantes)
   ✓ IsmaelSailema2.opus (12 variantes)
   ✓ IsmaelSailema3.opus (12 variantes)
   → 36 embeddings (shape: (36, 309))

👤 WilliamChimborazo
   ✓ WilliamChimborazo1.opus (12 variantes)
   ✓ WilliamChimborazo2.opus (12 variantes)
   ✓ WilliamChimborazo3.opus (12 variantes)
   → 36 embeddings (shape: (36, 309))

✓ Procesamiento completado
  AlisonSalas          - 36 embeddings (dim: 309)
  FreddyAlvarez        - 24 embeddings (dim: 309)
  IsmaelSailema        - 36 embeddings (dim: 309)
  WilliamChimborazo    - 36 embeddings (dim: 309)


In [43]:
# Guardar embeddings
pkl_file = OUTPUT_DIR / 'Embeddings_PyTorch.pkl'

datos = {
    'embeddings_por_usuario': embeddings_por_usuario,
    'usuarios': list(embeddings_por_usuario.keys()),
    'num_usuarios': len(embeddings_por_usuario),
    'embedding_dimension': list(embeddings_por_usuario.values())[0].shape[1] if embeddings_por_usuario else 0,
    'processor_config': {
        'sample_rate': SAMPLE_RATE,
        'n_mels': N_MELS,
        'n_mfcc': N_MFCC
    }
}

with open(pkl_file, 'wb') as f:
    pickle.dump(datos, f)

print(f"✓ Embeddings guardados: {pkl_file}")
print(f"  Tamaño: {pkl_file.stat().st_size / 1024:.1f} KB")

✓ Embeddings guardados: ProcessedData\Embeddings_PyTorch.pkl
  Tamaño: 159.8 KB


In [44]:
# Procesar audios de prueba
from pathlib import Path

test_audio_dir = Path('../RedNeuronal/AudiosPrueba')
test_output_dir = OUTPUT_DIR

if test_audio_dir.exists():
    print("\nProcesando audios de prueba...")
    print("=" * 70)
    
    test_embeddings = {}
    
    for item in sorted(test_audio_dir.iterdir()):
        if item.is_file() and item.suffix.lower() in ['.opus', '.wav', '.mp3']:
            usuario_name = item.stem
            audio, sr = processor.load_audio(item)
            
            if audio is not None:
                embedding = processor.extract_embedding(audio)
                if usuario_name not in test_embeddings:
                    test_embeddings[usuario_name] = []
                test_embeddings[usuario_name].append(embedding.numpy())
                print(f"  ✓ {item.name} -> {usuario_name}")
    
    if test_embeddings:
        test_embeddings = {k: np.array(v) for k, v in test_embeddings.items()}
        
        test_pkl = test_output_dir / 'Embeddings_Prueba_PyTorch.pkl'
        test_data = {
            'embeddings_por_usuario': test_embeddings,
            'usuarios': list(test_embeddings.keys()),
            'num_usuarios': len(test_embeddings),
            'embedding_dimension': list(test_embeddings.values())[0].shape[1]
        }
        
        with open(test_pkl, 'wb') as f:
            pickle.dump(test_data, f)
        
        print(f"\n✓ Test embeddings guardados: {test_pkl}")
else:
    print(f"\nNo encontrada: {test_audio_dir.absolute()}")


Procesando audios de prueba...
  ✓ Audio1I.opus -> Audio1I
  ✓ Audio2A.opus -> Audio2A
  ✓ Audio3R.opus -> Audio3R
  ✓ Audio4R.opus -> Audio4R
  ✓ Audio6W.opus -> Audio6W
  ✓ Audio7F.opus -> Audio7F

✓ Test embeddings guardados: ProcessedData\Embeddings_Prueba_PyTorch.pkl
